# SweepMeasurementConfig Usage

This notebook builds a sweep measurement configuration, then serializes it to a dictionary and JSON before restoring it.

In [1]:
import numpy as np

from measurement_config.models import (
    DataAcquisitionConfig,
    FrequencyConfig,
    ParameterSweepConfig,
    ParameterSweepContent,
    ParametricSequenceConfig,
    ParametricSequencePulseCommand,
    SweepMeasurementConfig,
)
from measurement_config.units import GHz, MHz, ns, s

## Create a configuration

Define sequence, frequency, data acquisition, and sweep parameters.

In [2]:
config = SweepMeasurementConfig(
    channel_list=[
        "Q0_qubit",
        "Q0_readout",
        "Q0_Q1_cross_resonance",
        "Q1_qubit",
        "Q1_readout",
    ],
    sequence=ParametricSequenceConfig(
        delta_time=2 * ns,
        variable_list=["duration"],
        command_list=[
            ParametricSequencePulseCommand(
                name="Gasussian",
                channel_list=["Q0_qubit"],
                argument_list=[100],
            ),
            ParametricSequencePulseCommand(
                name="TPCX",
                channel_list=["Q0_Q1_cross_resonance", "Q1_qubit"],
                argument_list=["duration*2"],
            ),
        ],
    ),
    frequency=FrequencyConfig(
        channel_to_frequency={
            "Q0_qubit": 4 * GHz,
            "Q0_readout": 6 * GHz,
            "Q1_qubit": 4.1 * GHz,
            "Q1_readout": 6.1 * GHz,
        },
        channel_to_frequency_shift={
            "Q0_qubit": 0 * GHz,
            "Q0_readout": 0 * GHz,
            "Q0_Q1_cross_resonance": 0 * GHz,
            "Q1_qubit": 0 * GHz,
            "Q1_readout": 0 * GHz,
        },
        channel_to_frequency_reference={"Q0_Q1_cross_resonance": "Q1_qubit"},
        keep_oscillator_relative_phase=True,
    ),
    data_acquisition=DataAcquisitionConfig(
        shot_count=1000,
        shot_repetition_margin=500 * ns,
        data_acquisition_delay=100 * ns,
        data_acquisition_duration=512 * ns,
        data_acquisition_timeout=2 * s,
        flag_average_shots=True,
        flag_average_waveform=True,
        channel_to_averaging_window={
            "Q0_readout": np.ones(shape=(10), dtype=complex),
        },
        delta_time=8 * ns,
        channel_to_averaging_time={
            "Q1_readout": 200 * ns,
        },
    ),
    sweep_parameter=ParameterSweepConfig(
        sweep_content_list={
            "tpcx_length": ParameterSweepContent(
                category="sequence_variable",
                sweep_target=["duration"],
                value_list=np.logspace(1, 3, 3),
            ),
            "qubit0_freq_sweep": ParameterSweepContent(
                category="frequency_shift",
                sweep_target=["Q0_qubit"],
                value_list=np.linspace(-2, 2, 5) * MHz,
            ),
            "qubit1_freq_sweep": ParameterSweepContent(
                category="frequency_shift",
                sweep_target=["Q1_qubit"],
                value_list=np.linspace(-2, 2, 5) * MHz,
            ),
        },
        sweep_axis=[["tpcx_length"], ["qubit0_freq_sweep", "qubit1_freq_sweep"]],
    ),
)
print(config)

channel_list=['Q0_qubit', 'Q0_readout', 'Q0_Q1_cross_resonance', 'Q1_qubit', 'Q1_readout'] sequence=ParametricSequenceConfig(delta_time=Time(2, 'ns'), variable_list=['duration'], command_list=[ParametricSequencePulseCommand(name='Gasussian', channel_list=['Q0_qubit'], argument_list=[100.0]), ParametricSequencePulseCommand(name='TPCX', channel_list=['Q0_Q1_cross_resonance', 'Q1_qubit'], argument_list=['duration*2'])]) frequency=FrequencyConfig(channel_to_frequency={'Q0_qubit': Frequency(4, 'GHz'), 'Q0_readout': Frequency(6, 'GHz'), 'Q1_qubit': Frequency(4.1, 'GHz'), 'Q1_readout': Frequency(6.1, 'GHz')}, channel_to_frequency_reference={'Q0_Q1_cross_resonance': 'Q1_qubit'}, channel_to_frequency_shift={'Q0_qubit': Frequency(0, 'GHz'), 'Q0_readout': Frequency(0, 'GHz'), 'Q0_Q1_cross_resonance': Frequency(0, 'GHz'), 'Q1_qubit': Frequency(0, 'GHz'), 'Q1_readout': Frequency(0, 'GHz')}, keep_oscillator_relative_phase=True) data_acquisition=DataAcquisitionConfig(shot_count=1000, shot_repetition_

## Serialization and Deserialization

Convert the configuration to a dictionary or JSON string, then restore it from JSON.

In [3]:
# To Dictionary
data_dict = config.to_dict()
print("Serialized dict:", data_dict)

# To JSON
json_str = config.to_json(indent=2)
print("JSON string:\n", json_str)

Serialized dict: {'channel_list': ['Q0_qubit', 'Q0_readout', 'Q0_Q1_cross_resonance', 'Q1_qubit', 'Q1_readout'], 'sequence': {'delta_time': {'units': [{'unit': 'SECOND', 'scale': 'NANO'}], 'real_value': 2.0, '__type__': 'tunits.Time'}, 'variable_list': ['duration'], 'command_list': [{'name': 'Gasussian', 'channel_list': ['Q0_qubit'], 'argument_list': [100.0]}, {'name': 'TPCX', 'channel_list': ['Q0_Q1_cross_resonance', 'Q1_qubit'], 'argument_list': ['duration*2']}]}, 'frequency': {'channel_to_frequency': {'Q0_qubit': {'units': [{'unit': 'HERTZ', 'scale': 'GIGA'}], 'real_value': 4.0, '__type__': 'tunits.Frequency'}, 'Q0_readout': {'units': [{'unit': 'HERTZ', 'scale': 'GIGA'}], 'real_value': 6.0, '__type__': 'tunits.Frequency'}, 'Q1_qubit': {'units': [{'unit': 'HERTZ', 'scale': 'GIGA'}], 'real_value': 4.1, '__type__': 'tunits.Frequency'}, 'Q1_readout': {'units': [{'unit': 'HERTZ', 'scale': 'GIGA'}], 'real_value': 6.1, '__type__': 'tunits.Frequency'}}, 'channel_to_frequency_reference': {'Q

In [4]:
# From Dictionary/JSON
restored_config = SweepMeasurementConfig.from_json(json_str)
print("Restored config:", restored_config)

Restored config: channel_list=['Q0_qubit', 'Q0_readout', 'Q0_Q1_cross_resonance', 'Q1_qubit', 'Q1_readout'] sequence=ParametricSequenceConfig(delta_time=Time(2.0, 'ns'), variable_list=['duration'], command_list=[ParametricSequencePulseCommand(name='Gasussian', channel_list=['Q0_qubit'], argument_list=[100.0]), ParametricSequencePulseCommand(name='TPCX', channel_list=['Q0_Q1_cross_resonance', 'Q1_qubit'], argument_list=['duration*2'])]) frequency=FrequencyConfig(channel_to_frequency={'Q0_qubit': Frequency(4.0, 'GHz'), 'Q0_readout': Frequency(6.0, 'GHz'), 'Q1_qubit': Frequency(4.1, 'GHz'), 'Q1_readout': Frequency(6.1, 'GHz')}, channel_to_frequency_reference={'Q0_Q1_cross_resonance': 'Q1_qubit'}, channel_to_frequency_shift={'Q0_qubit': Frequency(0.0, 'GHz'), 'Q0_readout': Frequency(0.0, 'GHz'), 'Q0_Q1_cross_resonance': Frequency(0.0, 'GHz'), 'Q1_qubit': Frequency(0.0, 'GHz'), 'Q1_readout': Frequency(0.0, 'GHz')}, keep_oscillator_relative_phase=True) data_acquisition=DataAcquisitionConfig(